In [5]:
import dynamiqs as dq
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.integrate import simpson
from scipy.ndimage import gaussian_filter

# Task A

## Utility Functions

In [8]:
#utility functions for affine distortions
#affinely distort wigner function
def affine_distort(wigner, alpha, b):
    return alpha*wigner+b

#add gaussian noise to wigner function
def add_gaussian_noise(wigner, sigma):
    noise = sigma*jax.random.normal(jax.random.key(0), wigner.shape)
    return jnp.add(wigner, noise)

def add_affine_gaussian_noise(wigner, alpha, b, sigma):
    #add afine first
    affine_wigner = affine_distort(wigner, alpha, b)
    #add gaussian noise
    return add_gaussian_noise(affine_wigner, sigma)

#calculate the norm of a wigner function
def integrate_wigner(wigner, x, y):
    return simpson(simpson(wigner, y), x)

#calculate the b value of the affine distortion
def recover_affine_offset(wigner):
    x_len, y_len = wigner.shape
    #assuming the corners don't make too much of a difference
    return 0.25*(jnp.mean(wigner[0,:]) + jnp.mean(wigner[x_len-1,:]) + jnp.mean(wigner[:,0]) + jnp.mean(wigner[:,y_len-1]))

#calculate the alpha value of the affine distortion
def recover_affine_scaling(zeroed_wigner, x, y):
    return integrate_wigner(zeroed_wigner, x, y)

#remove the affine distortion of a wigner function
def remove_affine_distortion(affine_wigner, x, y):
    b = recover_affine_offset(affine_wigner)
    zeroed_affined_wigner = affine_wigner - b
    alpha = recover_affine_scaling(wigner, x, y)
    return zeroed_affine_wigner/alpha

#run the simple affine denoising + gaussian filter pipeline
def remove_affine_gaussian_noise(noisy_wigner, x, y, sigma):
    #remove affine noise
    affine_less_wigner = remove_affine_distrotion(noisy_wigner, x, y)
    #apply gaussian filter
    filtered_wigner = gaussian_filter(affine_less_wigner, sigma)
    return filtered_wigner

## Benchmarking

In [9]:
#give an input state
#the benchmark will compare the reconstruction pipeline for noisy and simply denoised data
def affine_only_benchmark(reference_state, max_alpha, max_b):
    #setup the references/truths
    reference_dm = dq.todm(reference_state)
    x, y, reference_wigner = dq.wigner(reference_state)

    #add noise to the wigner function
    v_alpha = np.linspace(0, max_alpha, 5)
    v_b = np.linspace(0, max_b, 5)
    noisy_wigner_stack = add_affine_gaussian_noise(reference_wigner, v_alpha, v_b, 0)
    #reconstruct from noisy data
    noisy_rho_stack = fit ...
    noisy_fidelity = dq.fidelity

    #denoise
    denoised_wigner_stack = remove_affine_gaussian_noise(noisy_wigner_stack, x, y, 0)
    #reconstruct from denoised data
    


    return fidelity



SyntaxError: invalid syntax (3973140369.py, line 3)

In [10]:
def gauss_only_benchmark(reference_state, max_noise_sigma, max_filter_sigma):
    #setup the references/truths
    reference_dm = dq.todm(reference_state)
    x, y, reference_wigner = dq.wigner(reference_state)

    #add noise to the wigner function
    v_noise_sigma = np.linspace(0, max_noise_sigma, 5)
    noisy_wigner_stack = add_affine_gaussian_noise(reference_wigner, 1, 0, v_noise_sigma)

    #reconstruct from noisy data
    noisy_rho_stack = fit ...
    noisy_fidelity = dq.fidelity

    #denoise
    v_filter_sigma = np.linspace(0, max_filter_sigma, 5)
    denoised_wigner_stack = remove_affine_gaussian_noise(noisy_wigner_stack, x, y, v_filter_sigma)
    #reconstruct from denoised data

    
    
    
    return fidelity

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])